In [12]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider, fixed 
from IPython.display import display # Nécessaire pour afficher explicitement dans certains environnements

# Note: Ce code est conçu pour être exécuté dans un environnement Jupyter Notebook/Lab

def construire_hamiltonien_zigzag(k, N, t=1.0):
    """
    Construit la matrice Hamiltonienne H(k) pour un nanoruban zigzag.
    (Identique à la version précédente)
    """
    dim = 2 * N
    H = np.zeros((dim, dim), dtype=complex)
    gamma_k = 1 + np.exp(-1j * k)
    gamma_k_conj = np.conjugate(gamma_k)

    for m in range(N):
        idx_a = 2 * m
        idx_b = 2 * m + 1
        H[idx_a, idx_b] = -t * gamma_k
        H[idx_b, idx_a] = -t * gamma_k_conj

    for m in range(N - 1):
        idx_b_m = 2 * m + 1
        idx_a_m_plus_1 = 2 * (m + 1)
        H[idx_a_m_plus_1, idx_b_m] = -t
        H[idx_b_m, idx_a_m_plus_1] = -t

    return H

def calculer_bandes_zigzag(N, t=1.0, num_k_points=1000):
    """
    Calcule les énergies (valeurs propres) pour une plage de k de -pi à pi.

    Args:
        N (int): Largeur du ruban.
        t (float): Intégrale de saut.
        num_k_points (int): Nombre de points k à calculer entre -pi et pi.

    Returns:
        tuple: (k_values, eigenvalues)
            k_values (numpy.ndarray): Les valeurs de k utilisées.
            eigenvalues (numpy.ndarray): Tableau (num_k_points x 2N) des énergies.
    """
    # Modification: k va de -pi à pi
    k_values = np.linspace(-np.pi, np.pi, num_k_points)
    eigenvalues = np.zeros((num_k_points, 2 * N))

    for i, k in enumerate(k_values):
        Hk = construire_hamiltonien_zigzag(k, N, t)
        vals = np.linalg.eigvalsh(Hk)
        eigenvalues[i, :] = vals

    return k_values, eigenvalues

def tracer_bandes(k_values, eigenvalues, N, t=1.0):
    """
    Trace la structure de bandes de -pi à pi.

    Args:
        k_values (numpy.ndarray): Valeurs de k.
        eigenvalues (numpy.ndarray): Énergies correspondantes.
        N (int): Largeur du ruban (pour le titre).
        t (float): Valeur de t (pour l'échelle de l'axe y).
    """
    num_bands = eigenvalues.shape[1]
    # Afficher k en unités de pi/a (ici a=1)
    k_labels = k_values / np.pi

    plt.figure(figsize=(8, 6)) # Légèrement plus large pour la plage étendue
    for i in range(num_bands):
        plt.plot(k_labels, eigenvalues[:, i] / t, color='blue', linewidth=1.5)

    plt.xlabel(r'$k a / \pi$') # a=1 implicitement
    plt.ylabel(r'$E / t$')
    plt.title(f'Structure de Bandes - Nanoruban Zigzag (N = {N})')
    # Modification: xlim va de -1 à 1
    plt.xlim(-1, 1)
    min_E = np.min(eigenvalues / t)
    max_E = np.max(eigenvalues / t)
    plt.ylim(min(min_E - 0.2, -t*3.2), max(max_E + 0.2, t*3.2))
    plt.axhline(0, color='red', linestyle='--', linewidth=1, label='E = 0')
    # Ajouter des lignes verticales pour les points de haute symétrie 
    plt.axvline(x=-2/3, color='gray', linestyle=':', linewidth=0.8)
    plt.axvline(x=2/3, color='gray', linestyle=':', linewidth=0.8)
    plt.xticks([-1, -2/3, 0, 2/3, 1], [r'$-\pi$', r'$-2\pi/3$', '0', r'$2\pi/3$', r'$\pi$']) # Ticks personnalisés
    plt.grid(True, linestyle=':', alpha=0.7)
    plt.legend()
    plt.show() # Important pour afficher le graphique dans le contexte interactif

# --- Fonction pour l'interaction ---
def plot_interactive_bands(N, t=1.0, num_k_points=1000):
    """
    Fonction appelée par interact pour calculer et tracer les bandes pour une largeur N donnée.
    """
    k_vec, E_nk = calculer_bandes_zigzag(N, t=t, num_k_points=num_k_points)
    tracer_bandes(k_vec, E_nk, N, t=t)

# --- Création du Widget Interactif ---

# Définir le slider pour N
n_slider = IntSlider(
    min=2,         # Largeur minimale (au moins 2 pour avoir des bords)
    max=300,        # Largeur maximale
    step=1,        # Incrément
    value=200,      # Valeur initiale
    description='Largeur N:',
    continuous_update=False # Mettre à jour uniquement lorsque le slider est relâché
)

# Utiliser interact pour lier le slider à la fonction de tracé
interactive_plot = interact(
    plot_interactive_bands,
    N=n_slider,
    t=fixed(1.0), 
    num_k_points=fixed(1000) # Garder le nombre de points k fixe
)

interactive(children=(IntSlider(value=200, continuous_update=False, description='Largeur N:', max=300, min=2),…

# De l'Hamiltonien en Espace Réel à la Matrice en Espace k

Notre objectif est de calculer la structure de bandes électroniques d'un nanoruban de graphène de type zigzag. Le point de départ est l'hamiltonien de liaisons fortes (tight-binding) décrivant les sauts des électrons entre sites voisins sur le réseau.

## 1. Hamiltonien en Espace Réel

L'hamiltonien dans l'espace réel est donné dans l'Appendice B et se compose de deux termes principaux (en posant l'intégrale de saut $t$ et le paramètre de maille $a=1$ pour l'instant) :

$$
H = H_{\text{long}} + H_{\text{trans}}
$$


*   **Saut longitudinal ($H_{\text{long}}$)** : Décrit les sauts entre les sites A et B au sein de la même chaîne transverse $m$, mais entre des cellules unité $l$ et $l-1$, ainsi que les sauts entre A et B dans la *même* cellule unité $l$. En explicitant les conjugués hermitiens (C.h.), cela correspond à (adapté de B.1):

    $$
    H_{\text{long}} = -t \sum_{l=1}^{L_x} \sum_{m=1}^{N} \left[ a^\dagger_l(m) b_{l-1}(m) + b^\dagger_{l-1}(m) a_l(m) + a^\dagger_l(m) b_l(m) + b^\dagger_l(m) a_l(m) \right]
    $$

    où $a^\dagger_l(m)$ ($b^\dagger_l(m)$) crée un électron sur le site A (B) de la $m$-ième chaîne dans la $l$-ième cellule unité, et $a_l(m)$ ($b_l(m)$) l'annihile. $L_x$ est le nombre de cellules unité le long du ruban et $N$ est la largeur (nombre de chaînes).

*   **Saut transverse ($H_{\text{trans}}$)** : Décrit les sauts entre des chaînes voisines $m$ et $m+1$, mais au sein de la même cellule unité $l$. Cela correspond à (adapté de B.2) :

    $$
    H_{\text{trans}} = -t \sum_{l=1}^{L_x} \sum_{m=1}^{N-1} \left[ a^\dagger_l(m+1) b_l(m) + b^\dagger_l(m) a_l(m+1) \right]
    $$

Cet hamiltonien couple tous les sites du réseau. Pour obtenir la structure de bandes $E(k)$, qui relie l'énergie au vecteur d'onde $k$, nous devons exploiter la symétrie de translation le long de l'axe du ruban (direction $l$).

## 2. Transformation de Fourier

Nous appliquons une transformée de Fourier discrète le long de la direction $l$, en passant des opérateurs d'espace réel $a_l(m), b_l(m)$ aux opérateurs d'espace k $\alpha_k(m), \beta_k(m)$ :

$$
a_l(m) = \frac{1}{\sqrt{L_x}} \sum_k e^{ikl} \alpha_k(m) \quad ; \quad b_l(m) = \frac{1}{\sqrt{L_x}} \sum_k e^{ikl} \beta_k(m)
$$
$$
a^\dagger_l(m) = \frac{1}{\sqrt{L_x}} \sum_{k'} e^{-ik'l} \alpha^\dagger_{k'}(m) \quad ; \quad b^\dagger_l(m) = \frac{1}{\sqrt{L_x}} \sum_{k'} e^{-ik'l} \beta^\dagger_{k'}(m)
$$

En substituant ces expressions dans $H_{\text{long}}$ et $H_{\text{trans}}$ et en utilisant l'identité $\sum_{l=1}^{L_x} e^{i(k-k')l} = L_x \delta_{k,k'}$, l'hamiltonien se découple en une somme sur les vecteurs d'onde $k$ indépendants :

$$
H = \sum_k H(k)
$$

Après calcul (détaillé dans la dérivation précédente), on obtient l'expression de $H(k)$ qui n'agit que sur les degrés de liberté transverses (indice $m$) pour un $k$ donné :

$$
H(k) = -t \sum_{m=1}^{N} \left[ (1 + e^{-ik}) \alpha^\dagger_k(m) \beta_k(m) + (1 + e^{ik}) \beta^\dagger_k(m) \alpha_k(m) \right] - t \sum_{m=1}^{N-1} \left[ \alpha^\dagger_k(m+1) \beta_k(m) + \beta^\dagger_k(m) \alpha_k(m+1) \right]
$$

Nous avons défini $\gamma_k = 1 + e^{-ik}$. L'expression devient :
$$
H(k) = -t \sum_{m=1}^{N} \left[ \gamma_k \alpha^\dagger_k(m) \beta_k(m) + \gamma_k^* \beta^\dagger_k(m) \alpha_k(m) \right] - t \sum_{m=1}^{N-1} \left[ \alpha^\dagger_k(m+1) \beta_k(m) + \beta^\dagger_k(m) \alpha_k(m+1) \right]
$$

## 3. Construction de la Matrice $H(k)$

L'étape suivante consiste à représenter cet opérateur $H(k)$ sous forme de matrice pour pouvoir calculer ses valeurs propres numériquement, qui correspondent aux énergies $E(k)$.

La base naturelle pour cette matrice est l'ensemble des états localisés sur les sites A et B des $N$ chaînes transverses, pour un $k$ donné. Nous ordonnons cette base comme suit :
$| \psi_k \rangle = [ \alpha_k(1), \beta_k(1), \alpha_k(2), \beta_k(2), \dots, \alpha_k(N), \beta_k(N) ]^T$
où $\alpha_k(m)$ représente l'état sur le site A de la chaîne $m$ et $\beta_k(m)$ l'état sur le site B de la chaîne $m$.

La matrice $H(k)$ aura une dimension de $2N \times 2N$. Ses éléments sont déterminés par l'action de $H(k)$ sur les vecteurs de base. En utilisant la notation où $H_{ij}$ est l'élément de la ligne $i$ et de la colonne $j$ :

1.  **Termes $\sum_{m=1}^{N} [ \gamma_k \alpha^\dagger_k(m) \beta_k(m) + \gamma_k^* \beta^\dagger_k(m) \alpha_k(m) ]$ (Sauts intra-chaîne et longitudinaux combinés):**
    *   Le terme $-t \gamma_k \alpha^\dagger_k(m) \beta_k(m)$ connecte l'état $\beta_k(m)$ (colonne) à l'état $\alpha_k(m)$ (ligne). Il contribue à l'élément de matrice $H_{\alpha(m), \beta(m)} = -t \gamma_k$.
    *   Le terme $-t \gamma_k^* \beta^\dagger_k(m) \alpha_k(m)$ connecte l'état $\alpha_k(m)$ (colonne) à l'état $\beta_k(m)$ (ligne). Il contribue à l'élément de matrice $H_{\beta(m), \alpha(m)} = -t \gamma_k^*$.

2.  **Termes $\sum_{m=1}^{N-1} [ \alpha^\dagger_k(m+1) \beta_k(m) + \beta^\dagger_k(m) \alpha_k(m+1) ]$ (Sauts transverses inter-chaînes):**
    *   Le terme $-t \alpha^\dagger_k(m+1) \beta_k(m)$ connecte l'état $\beta_k(m)$ (colonne) à l'état $\alpha_k(m+1)$ (ligne). Il contribue à l'élément de matrice $H_{\alpha(m+1), \beta(m)} = -t$.
    *   Le terme $-t \beta^\dagger_k(m) \alpha_k(m+1)$ connecte l'état $\alpha_k(m+1)$ (colonne) à l'état $\beta_k(m)$ (ligne). Il contribue à l'élément de matrice $H_{\beta(m), \alpha(m+1)} = -t$.

**Indices dans le code Python :** En Python, les indices $m$ vont de `0` à `N-1`. Les indices de la matrice (lignes/colonnes) correspondant aux états sont :
*   $\alpha_k(m+1)$ (math) $\rightarrow$ indice `2*m` (Python)
*   $\beta_k(m+1)$ (math) $\rightarrow$ indice `2*m + 1` (Python)

La fonction `construire_hamiltonien_zigzag(k, N, t)` implémente précisément la construction de cette matrice $H(k)$ de taille $2N \times 2N$ en remplissant les éléments non nuls selon les règles ci-dessus. La diagonalisation numérique de cette matrice pour chaque $k$ nous donne les énergies $E_n(k)$ qui forment la structure de bandes.

In [13]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, fixed
from IPython.display import display
from joblib import Parallel, delayed

def eigenvalues_for_k(k, N, t=1.0):
    """
    Construit la matrice Hamiltonienne H(k) pour un nanoruban zigzag 
    et renvoie ses valeurs propres.
    
    Args:
        k (float): Paramètre de phase (vecteur d'onde).
        N (int): Largeur du ruban.
        t (float): Intensité du saut.
        
    Returns:
        np.ndarray: Tableau des valeurs propres calculées pour H(k).
    """
    dim = 2 * N
    H = np.zeros((dim, dim), dtype=complex)
    
    gamma_k = 1 + np.exp(-1j * k)
    gamma_k_conj = np.conjugate(gamma_k)
    
    # Remplissage des couplages internes
    for m in range(N):
        idx_a = 2 * m
        idx_b = 2 * m + 1
        H[idx_a, idx_b] = -t * gamma_k
        H[idx_b, idx_a] = -t * gamma_k_conj

    # Remplissage des couplages entre chaînes
    for m in range(N - 1):
        idx_b_m = 2 * m + 1
        idx_a_m_plus_1 = 2 * (m + 1)
        H[idx_a_m_plus_1, idx_b_m] = -t
        H[idx_b_m, idx_a_m_plus_1] = -t
        
    return np.linalg.eigvalsh(H)

def calculer_bandes_parallel(N, t=1.0, num_k_points=1000, n_jobs=-1):
    """
    Calcule de manière parallèle les valeurs propres de H(k) pour num_k_points 
    répartis entre -pi et pi.
    
    Args:
        N (int): Largeur du ruban.
        t (float): Intensité du saut.
        num_k_points (int): Nombre de points k.
        n_jobs (int): Nombre de cœurs à utiliser (n_jobs=-1 utilise tous les cœurs disponibles).
        
    Returns:
        tuple: (k_values, eigenvalues)
            - k_values (np.ndarray) : Tableau des valeurs k.
            - eigenvalues (np.ndarray) : Matrice (num_k_points x 2N) des valeurs propres.
    """
    k_values = np.linspace(-np.pi, np.pi, num_k_points)
    
    # Calcul parallèle sur chaque valeur de k
    eigenvalues = Parallel(n_jobs=n_jobs)(
        delayed(eigenvalues_for_k)(k, N, t) for k in k_values
    )
    
    # Conversion de la liste en tableau numpy
    eigenvalues = np.array(eigenvalues)
    return k_values, eigenvalues

def tracer_bandes(k_values, eigenvalues, N, t=1.0):
    """
    Trace la structure de bandes en fonction de k.
    
    Args:
        k_values (np.ndarray): Valeurs de k.
        eigenvalues (np.ndarray): Énergies associées.
        N (int): Largeur du ruban (pour le titre).
        t (float): Intensité du saut (pour l'échelle de l'axe y).
    """
    num_bands = eigenvalues.shape[1]
    k_labels = k_values / np.pi  # k en unités de pi
    
    plt.figure(figsize=(8, 6))
    for i in range(num_bands):
        plt.plot(k_labels, eigenvalues[:, i] / t, color='blue', linewidth=1.5)
    
    plt.xlabel(r'$k a / \pi$')
    plt.ylabel(r'$E / t$')
    plt.title(f'Structure de Bandes - Nanoruban Zigzag (N = {N})')
    plt.xlim(-1, 1)
    plt.axhline(0, color='red', linestyle='--', linewidth=1, label='E = 0')
    plt.axvline(x=-2/3, color='gray', linestyle=':', linewidth=0.8)
    plt.axvline(x=2/3, color='gray', linestyle=':', linewidth=0.8)
    plt.xticks([-1, -2/3, 0, 2/3, 1], [r'$-\pi$', r'$-2\pi/3$', '0', r'$2\pi/3$', r'$\pi$'])
    plt.grid(True, linestyle=':', alpha=0.7)
    plt.legend()
    plt.show()

def plot_interactive_bands(N, t=1.0, num_k_points=1000):
    """
    Fonction d'affichage interactif qui calcule et trace les bandes.
    
    Args:
        N (int): Largeur du ruban.
        t (float): Intensité du saut.
        num_k_points (int): Nombre de points k.
    """
    k_values, eigenvalues = calculer_bandes_parallel(N, t=t, num_k_points=num_k_points)
    tracer_bandes(k_values, eigenvalues, N, t=t)

# --- Création du widget interactif ---
n_slider = IntSlider(
    min=2,
    max=300,
    step=1,
    value=200,
    description='Largeur N:',
    continuous_update=False  # Mise à jour seulement lors du relâchement du slider
)

# Lancement de l'interactivité dans le notebook
interactive_plot = interact(
    plot_interactive_bands,
    N=n_slider,
    t=fixed(1.0),
    num_k_points=fixed(1000)
)


interactive(children=(IntSlider(value=200, continuous_update=False, description='Largeur N:', max=300, min=2),…

In [14]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider, fixed
from IPython.display import display

# Note: Ce code est conçu pour être exécuté dans un environnement Jupyter Notebook/Lab

def construire_hamiltonien_zigzag(k, N, t=1.0):
    """
    Construit la matrice Hamiltonienne H(k) pour un nanoruban zigzag.
    (Identique aux versions précédentes)
    """
    dim = 2 * N
    H = np.zeros((dim, dim), dtype=complex)
    gamma_k = 1 + np.exp(-1j * k)
    gamma_k_conj = np.conjugate(gamma_k)

    for m in range(N):
        idx_a = 2 * m
        idx_b = 2 * m + 1
        H[idx_a, idx_b] = -t * gamma_k
        H[idx_b, idx_a] = -t * gamma_k_conj

    for m in range(N - 1):
        idx_b_m = 2 * m + 1
        idx_a_m_plus_1 = 2 * (m + 1)
        H[idx_a_m_plus_1, idx_b_m] = -t
        H[idx_b_m, idx_a_m_plus_1] = -t

    return H

def calculer_bandes_zigzag(N, t=1.0, num_k_points=3000):
    """
    Calcule les énergies (valeurs propres) pour une plage de k de -pi à pi.
    (Identique à la version précédente)
    """
    k_values = np.linspace(-np.pi, np.pi, num_k_points)
    eigenvalues = np.zeros((num_k_points, 2 * N))

    for i, k in enumerate(k_values):
        Hk = construire_hamiltonien_zigzag(k, N, t)
        vals = np.linalg.eigvalsh(Hk)
        eigenvalues[i, :] = vals

    return k_values, eigenvalues

def calculer_et_tracer_bandes_dos(N, t=1.0, num_k_points=3000, num_energy_bins=500):
    """
    Calcule et trace la structure de bandes et la densité d'états (DOS).
    Utilise des subplots pour afficher les deux graphiques.
    """
    # 1. Calculer les bandes
    k_vec, E_nk = calculer_bandes_zigzag(N, t=t, num_k_points=num_k_points)

    # 2. Préparer les données pour la DOS
    all_energies = E_nk.flatten() # Collecter toutes les énergies calculées

    # 3. Calculer l'histogramme (DOS)
    # Utiliser density=True pour normaliser l'intégrale de l'histogramme à 1
    dos, bin_edges = np.histogram(all_energies / t, bins=num_energy_bins, density=True)
    # Calculer les centres des bins pour le tracé
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

    # 4. Tracer les résultats sur deux sous-graphiques
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6)) # 1 ligne, 2 colonnes

    # --- Tracé de la Structure de Bandes (ax1) ---
    num_bands = E_nk.shape[1]
    k_labels = k_vec / np.pi # k en unités de pi/a

    for i in range(num_bands):
        ax1.plot(k_labels, E_nk[:, i] / t, color='blue', linewidth=1.5)

    ax1.set_xlabel(r'$k a / \pi$')
    ax1.set_ylabel(r'$E / t$')
    ax1.set_title(f'Structure de Bandes (N = {N})')
    ax1.set_xlim(-1, 1)
    min_E = np.min(E_nk / t)
    max_E = np.max(E_nk / t)
    ax1.set_ylim(min(min_E - 0.2, -t*3.2), max(max_E + 0.2, t*3.2))
    ax1.axhline(0, color='red', linestyle='--', linewidth=1)
    ax1.axvline(x=-2/3, color='gray', linestyle=':', linewidth=0.8)
    ax1.axvline(x=2/3, color='gray', linestyle=':', linewidth=0.8)
    ax1.set_xticks([-1, -2/3, 0, 2/3, 1], [r'$-\pi$', r'$-2\pi/3$', '0', r'$2\pi/3$', r'$\pi$'])
    ax1.grid(True, linestyle=':', alpha=0.7)

    # --- Tracé de la Densité d'États (ax2) ---
    # On trace la DOS verticalement pour correspondre à l'axe d'énergie des bandes
    ax2.plot(dos, bin_centers, color='green', linewidth=1.5)
    ax2.set_xlabel(r'Densité d\'États (unités arb.)')
    ax2.set_ylabel(r'$E / t$')
    ax2.set_title(f'Densité d\'États (N = {N})')
    ax2.set_ylim(ax1.get_ylim()) # Assurer la même échelle d'énergie que les bandes
    ax2.grid(True, linestyle=':', alpha=0.7)
    ax2.axhline(0, color='red', linestyle='--', linewidth=1) # Ligne E=0 pour référence

    plt.tight_layout() # Ajuster l'espacement pour éviter les chevauchements
    plt.show()

# --- Création du Widget Interactif ---

# Définir le slider pour N
n_slider = IntSlider(
    min=2,
    max=275,
    step=1,
    value=150,
    description='Largeur N:',
    continuous_update=False # Mettre à jour uniquement lorsque le slider est relâché
)

# Utiliser interact pour lier le slider à la fonction de tracé combinée
interactive_plot = interact(
    calculer_et_tracer_bandes_dos,
    N=n_slider,
    t=fixed(1.0),
    num_k_points=fixed(3000),
    num_energy_bins=fixed(500)    # Nombre de bins pour l'histogramme DOS
)

# display(interactive_plot) # Généralement pas nécessaire dans les notebooks récents

interactive(children=(IntSlider(value=150, continuous_update=False, description='Largeur N:', max=275, min=2),…